In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [59]:
uncleaned_df = pd.read_csv("dataset.csv")
display(uncleaned_df)

,Country,Global Food Security Index (GFSI),Human Development Index (HDI),Corruption Index (CI),GDP per capita (GDP),Cost of Living Index (COL),Healthcare Index (HI)
0,Azerbaijan,62.3,0.756,30.0,4229.9,29.9,43.2
1,Bangladesh,50.0,0.632,26.0,2233.3,32.3,42.8
2,Cambodia,51.5,0.594,21.0,1577.9,47.5,-
3,China,69.3,0.761,42.0,10408.7,40,64.5
4,India,56.2,0.645,40.0,1913.2,24.6,67.1
5,Indonesia,59.5,0.718,37.0,3895.6,37.3,60.5
6,Japan,77.9,0.919,74.0,39986.9,83.3,81.1
7,Kazakhstan,70.8,0.825,38.0,9121.6,30.6,50.7
8,Laos,46.4,0.613,29.0,2593.4,-,-
9,Malaysia,67.9,0.810,51.0,10160.8,39.1,68.1


In [86]:
#Dataset cleaning

#Rename columns to acronyms
uncleaned_df = uncleaned_df.rename(columns={
    'Global Food Security Index (GFSI)': 'GFSI',
    'Human Development Index (HDI)': 'HDI',
    'Corruption Index (CI)': 'CI',
    'GDP per capita (GDP)': 'GDP',
    'Cost of Living Index (COL)': 'COL',
    'Healthcare Index (HI)': 'HI'
})

#Delete rows containing NaN values
df = uncleaned_df.replace("-", np.NaN).dropna()
df = df.drop(['Country'], axis=1)

df = df.astype(float) #convert all values in df to float

df


,GFSI,HDI,CI,GDP,COL,HI
0,62.3,0.756,30.0,4229.9,29.9,43.2
1,50.0,0.632,26.0,2233.3,32.3,42.8
3,69.3,0.761,42.0,10408.7,40.0,64.5
4,56.2,0.645,40.0,1913.2,24.6,67.1
5,59.5,0.718,37.0,3895.6,37.3,60.5
6,77.9,0.919,74.0,39986.9,83.3,81.1
7,70.8,0.825,38.0,9121.6,30.6,50.7
9,67.9,0.810,51.0,10160.8,39.1,68.1
11,53.0,0.602,33.0,1139.2,29.0,56.9
12,52.3,0.557,31.0,1322.3,22.0,60.6


In [87]:
# #Display the relationship between each predictor and the predicted value
# myplot = sns.pairplot(data = df) #TODO: NOT SURE WHY IT ONLY SHOWS SOME VALUES

In [88]:
columns = ["HDI", "CI", "GDP", "COL", "HI"]

# Set up feature and target datasets 
df_target = df[["GFSI"]]
df_features = df[columns]

row, col = df.shape

# display(df_target) 
# display(df_features)

In [89]:
def normalize_minmax(dfin, columns_mins = None, columns_maxs = None):
    dfin_copy = dfin.copy()

    if columns_mins == None:
        columns_mins = dfin_copy.min(axis=0)
        
    if isinstance(columns_mins, list):
        columns_mins = np.array(columns_mins)
    
    if columns_maxs == None:
        columns_maxs = dfin_copy.max(axis=0)

    if isinstance(columns_maxs, list):
        columns_maxs = np.array(columns_maxs)
    

    dfout = (dfin_copy - columns_mins) / (columns_maxs - columns_mins)

    return dfout, columns_mins, columns_maxs

In [90]:
def split_data(df_feature, df_target, random_state=None, test_size=0.5):
    
    indexes = df_feature.index
    n = int(len(indexes) * test_size)
    
    if random_state is not None:
        np.random.seed(random_state)
    
    test_indexes = np.random.choice(indexes, n, replace=False)
    train_indexes = list(set(indexes) - set(test_indexes))
    # train_indexes = filter(lambda x: x not in test_indexes, indexes)
    # train_indexes = [i for i in indexes if not in test_indexes]
    
    df_feature_train = df_feature.loc[train_indexes,:]
    df_target_train = df_target.loc[train_indexes,:]
    
    df_feature_test = df_feature.loc[test_indexes,:]
    df_target_test = df_target.loc[test_indexes,:]
    
    return df_feature_train, df_feature_test, df_target_train, df_target_test

In [92]:
# call normalize_minmax() to normalize df_features
data_norm, column_mins, column_maxs = normalize_minmax(df_features)

# call split_data() with seed of 100 and test size of 30%
df_features_train, df_features_test, df_target_train, df_target_train = split_data(data_norm, df_target, 100, 0.3)